<a href="https://www.kaggle.com/code/ahmedelgantiry/classification-model-human-activity-recognition?scriptVersionId=133850307" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/human-activity-recognition-with-smartphones/train.csv
/kaggle/input/human-activity-recognition-with-smartphones/test.csv


In [2]:
data=pd.read_csv("/kaggle/input/human-activity-recognition-with-smartphones/train.csv")

I will drop the "subject" column 

In [3]:
# I will drop the "subject "column because it will not add a predictive value to the model
data=data.drop(columns="subject",axis=1)

In [4]:
# confirm if the data is normalize 
data.iloc[:, :-1].min().value_counts()


-1.000000    466
-0.999996      2
-1.000000      2
-0.999893      2
-0.995377      2
            ... 
-0.943439      1
-0.998014      1
-0.999915      1
-0.902860      1
-0.976580      1
Length: 92, dtype: int64

In [5]:
data.iloc[:, :-1].max().value_counts()

1.000000    452
0.994731      2
0.646809      1
0.891736      1
0.702928      1
           ... 
0.979031      1
0.928416      1
0.848031      1
0.985138      1
0.478157      1
Length: 109, dtype: int64

In [6]:
data.Activity.value_counts()

LAYING                1407
STANDING              1374
SITTING               1286
WALKING               1226
WALKING_UPSTAIRS      1073
WALKING_DOWNSTAIRS     986
Name: Activity, dtype: int64

In [7]:
# using label encoder to transform the label to numbers from 1 to 5
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data['Activity'] = le.fit_transform(data.Activity)
data['Activity'].sample(5)

4439    0
2408    2
6345    0
1849    3
6800    2
Name: Activity, dtype: int64

In [8]:
#looking for corrlation columns and drop it
def correlation(dataset, threshold):
    col_corr = list() # Set of all the names of deleted columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if (corr_matrix.iloc[i, j] >= threshold) and (corr_matrix.columns[j] not in col_corr):
                colname = corr_matrix.columns[i] # getting the name of column
                col_corr.append(colname)
    return col_corr 

In [9]:

col_corr=correlation(data,.80)
len(col_corr)

642

In [10]:
col_corr.remove('Activity')

In [11]:
#drop the corrolated columns
data=data.drop(col_corr,axis=1)

In [12]:
data.shape

(7352, 198)

In [13]:
feature_cols = data.columns[:-1]
feature_cols

Index(['tBodyAcc-mean()-X', 'tBodyAcc-mean()-Y', 'tBodyAcc-mean()-Z',
       'tBodyAcc-std()-X', 'tBodyAcc-min()-X', 'tBodyAcc-energy()-Z',
       'tBodyAcc-entropy()-X', 'tBodyAcc-entropy()-Z',
       'tBodyAcc-arCoeff()-X,1', 'tBodyAcc-arCoeff()-X,2',
       ...
       'fBodyBodyGyroJerkMag-min()', 'fBodyBodyGyroJerkMag-maxInds',
       'fBodyBodyGyroJerkMag-meanFreq()', 'fBodyBodyGyroJerkMag-skewness()',
       'angle(tBodyAccMean,gravity)', 'angle(tBodyAccJerkMean),gravityMean)',
       'angle(tBodyGyroMean,gravityMean)',
       'angle(tBodyGyroJerkMean,gravityMean)', 'angle(Y,gravityMean)',
       'angle(Z,gravityMean)'],
      dtype='object', length=197)

In [14]:
# creating a validation data set
from sklearn.model_selection import StratifiedShuffleSplit

# Get the split indexes
strat_shuf_split = StratifiedShuffleSplit(n_splits=1, 
                                          test_size=0.3, 
                                          random_state=42)

train_idx, test_idx = next(strat_shuf_split.split(data[feature_cols], data.Activity))

# Create the dataframes
X_train = data.loc[train_idx, feature_cols]
y_train = data.loc[train_idx, 'Activity']

X_test  = data.loc[test_idx, feature_cols]
y_test  = data.loc[test_idx, 'Activity']

In [15]:
y_train.value_counts(normalize=True)

0    0.191411
2    0.186941
1    0.174893
3    0.166731
5    0.145939
4    0.134085
Name: Activity, dtype: float64

In [16]:
y_test.value_counts(normalize=True)

0    0.191296
2    0.186763
1    0.174977
3    0.166818
5    0.145966
4    0.134180
Name: Activity, dtype: float64

In [17]:

from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver='liblinear').fit(X_train, y_train)

In [18]:
from sklearn.linear_model import LogisticRegressionCV

# L1 regularized logistic regression
lr_l1 = LogisticRegressionCV(Cs=10, cv=4, penalty='l1', solver='liblinear').fit(X_train, y_train)

In [19]:
coefficients = list()

coeff_labels = ['lr', 'l1']
coeff_models = [lr, lr_l1]

for lab,mod in zip(coeff_labels, coeff_models):
    coeffs = mod.coef_
    coeff_label = pd.MultiIndex(levels=[[lab], [0,1,2,3,4,5]], 
                                 codes=[[0,0,0,0,0,0], [0,1,2,3,4,5]])
    coefficients.append(pd.DataFrame(coeffs.T, columns=coeff_label))

coefficients = pd.concat(coefficients, axis=1)

coefficients.sample(10)

lr                                                          l1  \
            0         1         2         3         4         5         0   
46  -0.087627  0.320864  0.251577  0.176375  0.885102 -1.176412 -0.264310   
121  0.136511 -0.321118  0.160630 -0.040778 -1.381513 -0.854108  0.000000   
29  -1.440757  1.165365  0.369932  0.780746 -2.352115  1.572874 -2.791325   
89   0.370040 -0.766069  0.034109 -0.480284 -0.543116  0.922721  0.000000   
90   0.145605 -1.034886  0.392136 -0.144658 -0.098501 -0.078315  0.000000   
67   0.043011 -0.701732  0.502108  0.408657  0.002699  0.661938  0.000000   
78  -0.152304 -0.146717  0.634649 -1.517076  0.176628  1.059091 -1.137428   
144 -0.084944  0.143843 -0.014271  1.493569 -0.788332 -0.345975  0.000000   
160  0.537733 -0.782642  0.624234 -0.169645 -0.160023 -0.570652  0.000000   
12  -0.002983 -0.062178  0.535661 -0.768671  0.881636 -0.264886  0.000000   

                                                       
            1         2         3         4         5  
46   0.000000  0.000000  0.005400  2.128774 -1.658947  
121 -0.361818  0.000000  0.000000 -2.300067 -3.845787  
29   2.641800 -0.214800  2.351237 -6.267941  1.381339  
89  -0.775072  0.000000 -1.990311 -1.375122  5.825415  
90  -2.805632  1.025096  0.000000  0.000000  0.000000  
67   0.000000  0.000000  0.971739  0.000000  0.000000  
78  -0.387230  0.801166 -5.746208  0.015125  6.321325  
144  0.000000  0.000000  3.549966 -1.680298  0.000000  
160 -0.860778  0.518977  0.000000 -1.875500 -2.270261  
12  -1.803482  4.277701 -3.691857  5.331772  0.000000

In [20]:
y_pred = list()
y_prob = list()

coeff_labels = ['lr', 'l1']
coeff_models = [lr, lr_l1, ]

for lab,mod in zip(coeff_labels, coeff_models):
    y_pred.append(pd.Series(mod.predict(X_test), name=lab))
    y_prob.append(pd.Series(mod.predict_proba(X_test).max(axis=1), name=lab))
    
y_pred = pd.concat(y_pred, axis=1)
y_prob = pd.concat(y_prob, axis=1)

y_pred.head()

,lr,l1
0,0,0
1,5,5
2,1,1
3,0,0
4,3,3


In [21]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score
from sklearn.preprocessing import label_binarize

metrics = list()
cm = dict()

for lab in coeff_labels:

    # Preciision, recall, f-score from the multi-class support function
    precision, recall, fscore, _ = score(y_test, y_pred[lab], average='weighted')
    
    # The usual way to calculate accuracy
    accuracy = accuracy_score(y_test, y_pred[lab])
    
    # ROC-AUC scores can be calculated by binarizing the data
    auc = roc_auc_score(label_binarize(y_test, classes=[0,1,2,3,4,5]),
              label_binarize(y_pred[lab], classes=[0,1,2,3,4,5]), 
              average='weighted')
    
    # Last, the confusion matrix
    cm[lab] = confusion_matrix(y_test, y_pred[lab])
    
    metrics.append(pd.Series({'precision':precision, 'recall':recall, 
                              'fscore':fscore, 'accuracy':accuracy,
                              'auc':auc}, 
                             name=lab))

metrics = pd.concat(metrics, axis=1)

In [22]:
metrics

,lr,l1
precision,0.973762,0.975203
recall,0.973708,0.975068
fscore,0.973675,0.975027
accuracy,0.973708,0.975068
auc,0.983973,0.984797


final testing on the original test set 

In [23]:
final_test=pd.read_csv("/kaggle/input/human-activity-recognition-with-smartphones/test.csv")

In [24]:
# same feature engneering as the validation dataset 
final_test=final_test.drop(col_corr,axis=1)

In [25]:
final_test=final_test.drop(columns="subject",axis=1)

In [26]:
# confirming the shape is the same as the validation dataset 
final_test.shape

(2947, 198)

In [27]:
final_test['Activity'] = le.fit_transform(final_test.Activity)

In [28]:
final_test_true_label=final_test["Activity"]

In [29]:
final_test_features=final_test.drop(columns="Activity",axis=1)

In [30]:
final_test_prediction=lr_l1.predict(final_test_features)

In [31]:
test_precision, test_recall, test_fscore, _ = score(final_test_prediction,final_test_true_label, average='weighted')

In [32]:
# final result of the model 
test_precision, test_recall, test_fscore, 

(0.9397065924982947, 0.9379029521547336, 0.9379343220678051)